<a href="https://colab.research.google.com/github/eneskosar/Image-Captioning/blob/main/BLEU_init_inject_model_lr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

dir = "/content/gdrive/My Drive/Colab Notebooks/Neural Network/Project v_working/"


Mounted at /content/gdrive/


# Project Func

In [ ]:
import h5py
import random
import math
import os
import requests
import numpy as np
from pickle import dump, load
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image as imm
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add, concatenate
import tensorflow as tf
from nltk.translate.bleu_score import corpus_bleu

# load train data
def read_train_data(fname):
    hf = h5py.File(fname,'r')
    train_cap = hf.get('train_cap')[()]
    train_imid = hf.get('train_imid')[()]
    word_code = hf.get('word_code')
    train_url = hf.get('train_url')
    return train_cap, train_imid, word_code, train_url

# load test data
def read_test_data(fname):
    hf = h5py.File(fname,'r')
    test_cap = hf.get('test_caps')[()]
    test_imid = hf.get('test_imid')[()]
    test_url = hf.get('test_url')
    return test_cap, test_imid, test_url

# this function takes url dataset and folder name
# downloads images in to folder
# if the data is training, it splits data into train-validation splits
# training image ids are written in train_img_ids.txt
# validation image ids are written in val_img_ids.txt
def download_imgs(img_urls,train_img_folder,tr_ts = 1):
    #train_img_folder = "train_imgs"
    if not os.path.exists(train_img_folder):
        os.mkdir(train_img_folder)
    broken_count = 0
    img_ids      = list()
    n_url        = img_urls.shape[0]
    counter      = 0
    for i in range(n_url):
        link = img_urls[i].decode()
        index = i + 1
        img_name = train_img_folder + "/" + str(index) + ".jpg"
        img_data = requests.get(link)
        if img_data.status_code == 200:
            img_ids.append(index)
            counter = counter + 1
            with open(img_name, 'wb') as handler:
                handler.write(img_data.content)
            if counter % 1000 == 0:
                print('Downloaded {} images'.format(counter))
        else:
            broken_count = broken_count + 1

    print('The number of broken links: {}'.format(broken_count))
    n_img = len(img_ids)
    print('The number of downloaded images: {}'.format(n_img))
    if tr_ts == 1:
        random.shuffle(img_ids)
        n_val = (0.15 * n_img)
        n_val = int(math.floor(n_val))
        val_imgs = img_ids[0:n_val]
        train_imgs = img_ids[n_val:n_img]
        np.savetxt('val_img_ids.txt',val_imgs,fmt='%i')
        np.savetxt('train_img_ids.txt',train_imgs,fmt='%i')
        return val_imgs, train_imgs
    else:
        np.savetxt('test_img_ids.txt',img_ids,fmt='%i')
        return img_ids

# this function takes word_code and returns two dicts
# 1: wtoid -> word to index, 2: idtow -> index to word
def load_word_code(wc):
    words = wc[()].dtype.names
    wtoid, idtow = dict(), dict()
    i = 0
    for w in words:
        wtoid[w] = int(wc[(0)][i])
        idtow[int(wc[(0)][i])] = w
        i = i +1
    return wtoid, idtow

# this function prepare data for model
# returns image features, captions and corresponding y labels
def prepare_data(img_inds, img_feats, caps,im_ids, w_id):
    img_data  = []
    desc_data = []
    label     = []
    for imid in img_inds:
        im_feat   = img_feats[str(imid)]
        caps_inds = im_ids == imid
        im_caps   = caps[caps_inds,:]
        # for each caption
        for ic in range(im_caps.shape[0]):
            # for each word in a caption
            cap_length = im_caps.shape[1]
            x = np.zeros((1,cap_length))
            for iw in range(cap_length-1):
                x[0,0:(iw+1)] = im_caps[ic,0:(iw+1)]
                y = np.zeros((1,len(w_id.keys())))
                y_ind = im_caps[ic,(iw+1)]
                y[0,y_ind] = 1
                img_data.append(im_feat)
                desc_data.append(x)
                label.append(y)

    img_data  = np.array(img_data)
    desc_data = np.array(desc_data)
    label     = np.array(label)
    img_data = np.reshape(img_data, (img_data.shape[0],img_data.shape[2]))
    desc_data = np.reshape(desc_data, (desc_data.shape[0],desc_data.shape[2]))
    label = np.reshape(label, (label.shape[0],label.shape[2]))
    return img_data, desc_data, label

# returns image features, captions and corresponding y labels
def prepare_data2(im_feat, caps, w_id):
    img_data  = []
    desc_data = []
    label     = []
    # for each caption
    for ic in range(caps.shape[0]):
        # for each word in a caption
        cap_length = caps.shape[1]
        #x = np.zeros((1,cap_length),dtype=int)
        for iw in range(cap_length-1):
            x = np.zeros((1,cap_length),dtype=int)
            y_ind = caps[ic,(iw+1)]
            x[0,0:(iw+1)] = caps[ic,0:(iw+1)]
            y = np.zeros((1,len(w_id.keys())))
            y[0,y_ind] = 1
            img_data.append(im_feat)
            desc_data.append(x)
            label.append(y)
            if y_ind == 2:
                break

    img_data  = np.array(img_data)
    desc_data = np.array(desc_data)
    label     = np.array(label)
    img_data = np.reshape(img_data, (img_data.shape[0],img_data.shape[2]))
    desc_data = np.reshape(desc_data, (desc_data.shape[0],desc_data.shape[2]))
    label = np.reshape(label, (label.shape[0],label.shape[2]))
    return img_data, desc_data, label


def data_loader(img_inds,caps, img_feats, im_ids, w_id):
    # for each image iteratively
    while 1:
        for imid in img_inds:
            im_feat   = img_feats[str(imid)]
            # get a single image feature
            #img = img_feats[str(key)][0]
            caps_inds = im_ids == imid
            im_caps   = caps[caps_inds,:]
            img_data, desc_data, label = prepare_data2(im_feat, im_caps, w_id)
            yield [img_data, desc_data], label

def data_loader2(caps, img_feats, w_id):
    # for each image iteratively
    while 1:
        for key, caption in caps.items():
            # get a single image feature
            img = img_feats[str(key)][0]
            img_data, desc_data, label = prepare_data2(img, caption, w_id)
            yield [img_data, desc_data], label


# extract features from each photo in the path by using VGG16 models
def VGG16_features(path,features_file):
    model = VGG16()
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    im_features = dict()
    imgs_fname = os.listdir(path)
    for im_name in imgs_fname:
        # load image into array
        im = imm.load_img((path + im_name), target_size=(224, 224))
        im = imm.img_to_array(im)
        im = im.reshape((1, im.shape[0], im.shape[1], im.shape[2]))
        # preprocess the image as suggested
        image = vgg_preprocess_input(im)
        # forward-pass to get image features
        feature = model.predict(image, verbose=0)
        # get image id
        index = im_name.split('.')[0]
        # store feature
        im_features[index] = feature

    dump(im_features, open(features_file, 'wb'))
    return im_features

def InceptionV3_features(path,features_file):
    model = tf.keras.applications.InceptionV3(weights='imagenet')
    model = Model(model.input, model.layers[-2].output)
    im_features = dict()
    imgs_fname = os.listdir(path)
    for im_name in imgs_fname:
        # load image into array
        im = imm.load_img((path + im_name), target_size=(299, 299))
        im = imm.img_to_array(im)
        im = np.expand_dims(im, axis=0)
        # preprocess the image as suggested
        image = tf.keras.applications.inception_v3.preprocess_input(im)
        # forward-pass to get image features
        image = np.reshape(image, image.shape[1])
        feature = model.predict(image, verbose=0)
        # get image id
        index = im_name.split('.')[0]
        # store feature
        im_features[index] = feature

    dump(im_features, open(features_file, 'wb'))
    return im_features

def VGG16_features2(path,features_file):
    model = VGG16()
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    im_features = dict()
    imgs_fname = os.listdir(path)
    for im_name in imgs_fname:
        # load image into array
        try:
            im = imm.load_img((path + im_name), target_size=(224, 224))
            im = imm.img_to_array(im)
            im = im.reshape((1, im.shape[0], im.shape[1], im.shape[2]))
            # preprocess the image as suggested
            image = vgg_preprocess_input(im)
            # forward-pass to get image features
            feature = model.predict(image, verbose=0)
            # get image id
            index = im_name.split('.')[0]
            # store feature
            im_features[index] = feature
        except:
            print(im_name)

    dump(features, open(features_file, 'wb'))
    return im_features


def captionVGG_addition_model(drop,emb_size,l_rate):
    img_branch = Input(shape=(4096,))
    im_net = Dropout(drop)(img_branch)
    im_net = Dense(emb_size, activation='relu')(im_net)

    word_branch = Input(shape=(17,))
    word_net = Dropout(drop)(Embedding(1004, emb_size, mask_zero=True)(word_branch))
    word_net = LSTM(emb_size)(word_net)

    adding_layer = add([im_net, word_net])
    decode_layer = Dense(emb_size, activation='relu')(adding_layer)
    outputs = Dense(1004, activation='softmax')(decode_layer)

    model = Model(inputs=[img_branch, word_branch], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=float(l_rate), beta_1=0.9, beta_2=0.999, amsgrad=False))
    # optimizer=keras.optimizers.Adam(lr=float(l_rate), beta_1=0.9, beta_2=0.999, amsgrad=False)
    #print(model.summary())
    #plot_model(model, to_file='model.png', show_shapes=True)
    return model

def captionVGG_concat_model(drop,emb_size):
    img_branch = Input(shape=(4096,))
    im_net = Dropout(drop)(img_branch)
    im_net = Dense(emb_size, activation='relu')(im_net)

    word_branch = Input(shape=(17,))
    word_net = Dropout(drop)(Embedding(1004, emb_size, mask_zero=True)(word_branch))
    word_net = LSTM(emb_size)(word_net)

    concat_layer = concatenate([im_net, word_net])
    decode_layer = Dense(emb_size*2, activation='relu')(concat_layer)
    outputs = Dense(1004, activation='softmax')(decode_layer)

    model = Model(inputs=[img_branch, word_branch], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    #print(model.summary())
    #plot_model(model, to_file='model.png', show_shapes=True)
    return model

def captionVGG_inject_addition_model(drop,emb_size):
    img_branch = Input(shape=(4096,))
    im_net = Dropout(drop)(img_branch)
    im_net = Dense(emb_size, activation='relu')(im_net)

    word_branch = Input(shape=(17,))
    word_net = Dropout(drop)(Embedding(1004, emb_size, mask_zero=True)(word_branch))
    #word_net = LSTM(256)(word_net)

    adding_layer = add([im_net, word_net])
    rnn_layer = LSTM(256)(adding_layer)
    decode_layer = Dense(emb_size, activation='relu')(rnn_layer)
    outputs = Dense(1004, activation='softmax')(decode_layer)

    model = Model(inputs=[img_branch, word_branch], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    #print(model.summary())
    #plot_model(model, to_file='model.png', show_shapes=True)
    return model

def captionVGG_inject_concat_model(drop,emb_size):
    img_branch = Input(shape=(4096,))
    im_net = Dropout(drop)(img_branch)
    im_net = Dense(emb_size, activation='relu')(im_net)

    word_branch = Input(shape=(17,))
    word_net = Dropout(drop)(Embedding(1004, emb_size, mask_zero=True)(word_branch))
    word_net = Dense(emb_size, activation='relu')(word_net)

    concat_layer = concatenate([im_net, word_net])
    rnn_layer = LSTM(concat_layer.shape[1])(word_net)
    decode_layer = Dense(emb_size*2, activation='relu')(concat_layer)
    outputs = Dense(1004, activation='softmax')(decode_layer)

    model = Model(inputs=[img_branch, word_branch], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    #print(model.summary())
    #plot_model(model, to_file='model.png', show_shapes=True)
    return model

def img_to_caption(model, im_feat, idtow, im_caps):
    s = np.array([[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])
    for i in range(16):
        pred = model.predict([im_feat,s], verbose=0)
        wid = np.argmax(pred)
        s[0,(i+1)] = wid

    caption_pred = vec_to_caption(s, idtow, 1)
    word_caps = [vec_to_caption(im_caps[x,:], idtow, 0) for x in range(im_caps.shape[0])]
    return caption_pred, word_caps

def vec_to_caption(vec, idtow,pr=1):
    caption = []
    if pr == 1:
        for i in range(17):
            if vec[0,i] == 2:
                break
            caption.append(idtow[vec[0,i]])
    else:
        for i in range(17):
            if vec[i] == 2:
                break
            caption.append(idtow[vec[i]])

    return caption

def calculate_Bleu(model, img_feats, idtow, caps, im_ids):
    captions = []
    predicted_captions = []
    counter = 0
    #for imid in img_feats.keys():
    im_count = np.unique(im_ids).shape[0]
    im_count = [300, 400, 600, 700, 750, 800, 900, 100]
    for imid in im_count:
        try:
            im_feat   = img_feats[str(imid+1)]
            caps_inds = im_ids == (int(imid))
            im_caps   = caps[caps_inds,:]
            caption_pred, word_caps = img_to_caption(model, im_feat, idtow, im_caps)
            captions.append(word_caps)
            predicted_captions.append(caption_pred)
            counter = counter + 1
            print("==============================")
            print(imid)
            print(word_caps)
            print(caption_pred)
            if counter == 10000:
                break
            if counter % 100 == 0:
                print(counter)
        except:
            pass

    blue_score = corpus_bleu(captions, predicted_captions)
    return blue_score


'''def generate_desc(model, wtoid, idtow,photo, max_length=17):
    # seed the generation process
    in_text = ['x_START_']
    # iterate over the whole length of the sequence
    for i in range(max_length):
        # integer encode input sequence
        sequence = [wtoid[x] for x in in_text]
        # pad input
        sequence = pad_sequences([sequence], maxlen=max_length)
        # predict next word
        yhat = model.predict([photo,sequence], verbose=0)
        # convert probability to integer
        yhat = np.argmax(yhat)
        # map integer to word
        word = idtow[yhat]
        # stop if we cannot map the word
        if word is None:
            break
        # append as input for generating the next word
        in_text.append(word)
        # stop if we predict the end of the sequence
    return in_text'''


"def generate_desc(model, wtoid, idtow,photo, max_length=17):\n    # seed the generation process\n    in_text = ['x_START_']\n    # iterate over the whole length of the sequence\n    for i in range(max_length):\n        # integer encode input sequence\n        sequence = [wtoid[x] for x in in_text]\n        # pad input\n        sequence = pad_sequences([sequence], maxlen=max_length)\n        # predict next word\n        yhat = model.predict([photo,sequence], verbose=0)\n        # convert probability to integer\n        yhat = np.argmax(yhat)\n        # map integer to word\n        word = idtow[yhat]\n        # stop if we cannot map the word\n        if word is None:\n            break\n        # append as input for generating the next word\n        in_text.append(word)\n        # stop if we predict the end of the sequence\n    return in_text"

# Main

In [ ]:
import numpy as np
from pickle import dump, load
from tensorflow.keras.models import Model
import os
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
os.environ["CUDA_VISIBLE_DEVICES"]= "0"

num_cores = 8

GPU = True
if GPU:
    num_GPU = 1
    num_CPU = 1
else:
    num_CPU = 2
    num_GPU = 0

config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=num_cores,
                        inter_op_parallelism_threads=num_cores,
                        allow_soft_placement=True,
                        device_count = {'CPU' : num_CPU,
                                        'GPU' : num_GPU}
                       )

session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

train_cap, train_imid, word_code, train_url = read_train_data(dir + 'eee443_project_dataset_train.h5')
test_cap, test_imid, test_url = read_test_data(dir + 'eee443_project_dataset_test.h5')

test_img_path = dir + "test_im/"

###val_imgs = np.loadtxt("val_img_ids.txt",dtype=np.int)
###train_imgs = np.loadtxt("train_img_ids.txt",dtype=np.int)



wtoid, idtow = load_word_code(word_code)
features_file_test = dir + 'vgg16_features_test.pkl'
features_file = dir + 'vgg16_features.pkl'

#comment below after 
#vgg_im_features_test = VGG16_features(test_img_path, features_file_test)


    

In [ ]:
features_file_test = dir + 'vgg16_features_test.pkl'

vgg_im_features_test = load(open(features_file_test, 'rb'))
drop = 0.5
emb_size = 128
#model = captionVGG_addition_model(drop,emb_size)
emb_sizes = [256, 512, 128]

for emb_size in emb_sizes:
    
  #modelName = "/mnt/hikuru_backup/NN_PROJECT_FILES/models/model_addition_emb_size_" + str(emb_size) + ".h5"
  i = 1
  modelName = dir + "Models2/init_inject_embedSize" + str(emb_size) + "lr0001" + ".h5"
  #model.load_weights(modelName)

  model = tf.keras.models.load_model(modelName)
  print("PREDICTION STARTS")
  bleu_score = calculate_Bleu(model, vgg_im_features_test, idtow, test_cap, test_imid)
  print("BLEU SCORE DONE")
  print(bleu_score)

  '''ind = 1
  ii = test_imid == ind
  cc = test_cap[ii,]
  im_feat = np.array(vgg_im_features_test[str(ind)])

  print(cc)

  s = np.array([[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])
  for i in range(16):
      pred = model.predict([im_feat,s], verbose=0)
      wid = np.argmax(pred)
      #print(wid)
      s[0,(i+1)] = wid

  print("HERE")
  print(s)
  ss = generate_desc(model, wtoid, idtow, im_feat, max_length=17)
  print("HERE 2")
  print(ss)

  for i in range(s.shape[1]):
      print(idtow[s[0,i]])'''



PREDICTION STARTS
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
BLEU SCORE DONE
0.26761775371693436
PREDICTION STARTS
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800


In [ ]:
features_file_test = dir + 'vgg16_features_test.pkl'

vgg_im_features_test = load(open(features_file_test, 'rb'))
drop = 0.5
emb_size = 128
#model = captionVGG_addition_model(drop,emb_size)
emb_sizes = [256, 512, 128]

for emb_size in emb_sizes:
    
  #modelName = "/mnt/hikuru_backup/NN_PROJECT_FILES/models/model_addition_emb_size_" + str(emb_size) + ".h5"
  i = 1
  modelName = dir + "Models2/init_inject_embedSize" + str(emb_size) + "lr0001" + ".h5"
  #model.load_weights(modelName)

  model = tf.keras.models.load_model(modelName)
  print("PREDICTION STARTS")
  bleu_score = calculate_Bleu(model, vgg_im_features_test, idtow, test_cap, test_imid)
  print("BLEU SCORE DONE")
  print(bleu_score)

  '''ind = 1
  ii = test_imid == ind
  cc = test_cap[ii,]
  im_feat = np.array(vgg_im_features_test[str(ind)])

  print(cc)

  s = np.array([[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])
  for i in range(16):
      pred = model.predict([im_feat,s], verbose=0)
      wid = np.argmax(pred)
      #print(wid)
      s[0,(i+1)] = wid

  print("HERE")
  print(s)
  ss = generate_desc(model, wtoid, idtow, im_feat, max_length=17)
  print("HERE 2")
  print(ss)

  for i in range(s.shape[1]):
      print(idtow[s[0,i]])'''



PREDICTION STARTS
400
[['x_START_', 'a', 'large', 'x_UNK_', 'of', 'plants', 'x_UNK_', 'in', 'a', 'forest'], ['x_START_', 'there', 'are', 'a', 'lot', 'of', 'bushes', 'in', 'the', 'field'], ['x_START_', 'a', 'forest', 'with', 'lots', 'of', 'small', 'green', 'plants'], ['x_START_', 'new', 'plants', 'are', 'x_UNK_', 'up', 'from', 'the', 'forest', 'floor'], ['x_START_', 'a', 'couple', 'of', 'different', 'plants', 'in', 'the', 'woods']]
['x_START_', 'a', 'x_UNK_', 'of', 'green', 'leaves', 'on', 'a', 'tree']
600
[['x_START_', 'there', 'are', 'many', 'baseball', 'players', 'x_UNK_', 'each', 'other', 'on', 'the', 'field'], ['x_START_', 'baseball', 'players', 'x_UNK_', 'each', 'other', 'on', 'a', 'baseball', 'field'], ['x_START_', 'baseball', 'players', 'are', 'x_UNK_', 'with', 'each', 'other', 'after', 'a', 'x_UNK_'], ['x_START_', 'a', 'number', 'of', 'baseball', 'players', 'on', 'a', 'field', 'with', 'each', 'other'], ['x_START_', 'a', 'group', 'of', 'men', 'on', 'a', 'baseball', 'field', 'giv